In [ ]:
import pandas as pd

In [ ]:
df1 = pd.read_excel('Company X - Order Report.xlsx')
df1 = df1.rename(columns={'ExternOrderNo': 'Order ID'})
df1.head()

In [ ]:
df2 = pd.read_excel('Company X - Pincode Zones.xlsx')
df2['Zone'] = df2['Zone'].str.upper()
df2.head()

In [ ]:
df3 = pd.read_excel('Company X - SKU Master.xlsx')
df3.head()

In [ ]:
df4 = pd.read_excel('Courier Company - Invoice.xlsx')
df4['Zone'] = df4['Zone'].str.upper()
df4.head()

In [ ]:
df5 = pd.read_excel('Courier Company - Rates.xlsx')
df5.head()

In [ ]:
X = pd.merge(df1, df3, on='SKU', how='outer')
X['Total Weight (KG)'] = (X['Order Qty']*X['Weight (g)'])/1000
X.head()

In [ ]:
Courier_company = pd.merge(df4, df5, on='Zone', how='outer')
Courier_company = pd.merge(Courier_company, df2, on=['Warehouse Pincode', 'Customer Pincode',	'Zone'], how='outer')
temp = X[['Order ID','Payment Mode','Total Weight (KG)']]
Courier_company = pd.merge(Courier_company, temp, on='Order ID', how='outer')
Courier_company.head()

In [ ]:
Courier_company.info()

In [ ]:
Courier_company = Courier_company.dropna()

In [ ]:

df = Courier_company

def calculate_courier_charges(row):
    if "Forward and RTO charges" in row["Type of Shipment"]:
        forward_charges = row["Forward Fixed Charge"]
        additional_weight_slab_charge = row["Forward Additional Weight Slab Charge"]
        rto_charges = row["RTO Fixed Charge"]
        rto_additional_weight_slab_charge = row["RTO Additional Weight Slab Charge"]
    else:
        forward_charges = row["Forward Fixed Charge"]
        additional_weight_slab_charge = row["Forward Additional Weight Slab Charge"]
        rto_charges = 0
        rto_additional_weight_slab_charge = 0

    weight_slabs = df["Weight Slabs"].tolist()
    charged_weight = row["Charged Weight"]
    zone = row["Zone"]
    total_charges = 0

    for i in range(len(weight_slabs)):
        if i == 0:
            fixed_charge = forward_charges if "Forward charges" in row["Type of Shipment"] else rto_charges
            slab_weight = float(weight_slabs[i])
            total_charges += fixed_charge
        else:
            additional_slab_weight = float(weight_slabs[i]) - float(weight_slabs[i-1])
            additional_slab_charge = additional_weight_slab_charge if "Forward charges" in row["Type of Shipment"] else rto_additional_weight_slab_charge
            total_charges += (additional_slab_charge * additional_slab_weight)

    return total_charges

df["Courier Charges"] = df.apply(calculate_courier_charges, axis=1)

def calculate_cod_charges(row):
    if row["Payment Mode"] == "Prepaid":
        return 0
    else:
        order_amount = row["Billing Amount (Rs.)"]
        if order_amount <= 300:
            return 15
        else:
            return 0.05 * order_amount

df["COD Charges"] = df.apply(calculate_cod_charges, axis=1)
df["Total Charges"] = df["Courier Charges"] + df["COD Charges"]


In [ ]:
result = df[['Order ID','AWB Code']]
result['Total Weight - X (KG)'] = df['Total Weight (KG)']
result['Weight Slab - X (KG)'] = df['Weight Slabs']
result['Total Weight - Courier Company (KG)'] = df['Total Weight (KG)']+df['Weight Slabs']
result['Weight Slab Charged - Courier Company (KG)'] = df['Charged Weight']
result['Delivery Zone - X'] = df['Zone']
result['Delivery Zone Charged - Courier Company'] = df['Courier Charges']
result['Expected Charge - X (RS.)'] = df['Total Charges']
result['Charges Billed - Courier Company (RS.)'] = df['Billing Amount (Rs.)']
result['Differences - Expected Charges & Billed Charges'] = df['Billing Amount (Rs.)']-df['Total Charges']

In [ ]:
result.info()

In [ ]:
correct_crg = result.index[result['Expected Charge - X (RS.)'] == result['Charges Billed - Courier Company (RS.)']].tolist()
over_crg = result.index[result['Expected Charge - X (RS.)'] < result['Charges Billed - Courier Company (RS.)']].tolist()
under_crg = result.index[result['Expected Charge - X (RS.)'] > result['Charges Billed - Courier Company (RS.)']].tolist()

equal_sum = result.loc[correct_crg, 'Differences - Expected Charges & Billed Charges'].sum()
over_sum = result.loc[over_crg, 'Differences - Expected Charges & Billed Charges'].sum()
under_sum = result.loc[under_crg, 'Differences - Expected Charges & Billed Charges'].sum()

In [ ]:
data = {
    ' ': ['Total orders where X has been correctly charged', 'Total orders where X has been overcharged', 'Total Orders where X has been undercharged'],
    'Count': [len(correct_crg), len(over_crg), len(under_crg)],
    'Amount (Rs.)': [round(equal_sum,2), round(over_sum,2), round(under_sum,2)]
}
summary = pd.DataFrame(data)

In [ ]:

with pd.ExcelWriter('Output.xlsx') as writer:
    result.to_excel(writer, sheet_name='Calculations', index=False)
    summary.to_excel(writer, sheet_name='Summary', index=False)